# Day 25: 猜字遊戲

語言模型可以以字詞為單位作N-gram，也可以以字母為單位。為了讓大家對語言模型有個更清楚的理解，我們用英文猜字遊戲Hangman為範例來學習。

## 猜字遊戲Hangman

<a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a>的玩法是透過一個人寫下一個字，讓另一個人來猜。對方一次會猜一個英文字母，猜對的話就把字母寫到正確的格子上，猜錯的話則會記錄下來。若是猜錯的次數達到一個數字，則猜字的一方輸。

這裡我們先開發一個簡易版：

In [1]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        secret_word是將要被猜的字、guesser是我們之後會陸續寫進來的猜字模型（真人猜字或AI）、max_mistakes是最多可以錯誤的次數、
        verbose為True時表示互動性猜字（AI猜字時可以改False）、guesser_args是一個keyword argument。
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word) # 把要被猜的字轉成暗文
    guessed = set()
    if verbose:
        print("開始猜字遊戲，提示：", ' '.join(mask), '長度為', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("你還有", (max_mistakes-mistakes), "次機會。")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('你猜了：', guess)
        if guess in guessed:
            if verbose:
                print('這個字母已經猜過了！')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('猜對了，', ' '.join(mask))
            else:
                if verbose:
                    print('抱歉，再猜猜')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('恭喜你贏了！')
            return mistakes
        
    if verbose:
        print('沒有機會了，正確字是', secret_word)    
    return mistakes

這裡我們先寫一個人機互動猜字版：

In [2]:
def human(mask, guessed, **kwargs):
    """
    可以手動遊玩
    """
    print('請輸入你要猜的字：')
    return input().lower().strip()

interactive = True

試玩遊戲：

In [ ]:
if interactive:
    hangman('algorithm', human, 8, True)

開始猜字遊戲，提示： _ _ _ _ _ _ _ _ _ 長度為 9
你還有 8 次機會。
請輸入你要猜的字：


## 準備訓練和測試集

開始寫我們的AI猜字之前，要先準備訓練集和測試集。從[這篇文章](http://datagenetics.com/blog/april12012/index.html)中可以看到，照著統計學來猜字是個不錯的做法，因此我們也自己來訓練一組模型，照著出現的頻率為順序進行猜字。

我們使用之前用過的*Brown Corpus*來訓練AI猜字演算法，為了刻意增加猜字的難度，AI在測試時的字和訓練的字會是不一樣的(把訓練集和測試集拆開)，如此才能訓練出更通用的模型。

我們使用 `nltk.corpus.Brown` 裡的 `words()` 方法，從中將非英文字母的文字去除，同時將每個字小寫化。之後，我們用 `numpy.random.shuffle` 來隨機刷取文集中的字，以分成訓練集和測試集。測試集裡有1000字，剩下的字都會到訓練集裡面。

In [3]:
from nltk.corpus import brown
import numpy as np

np.random.seed(12345)

# word_set 儲存Brown Corpus中所有獨特的字型
word_set = []
# test_set 儲存1000個字的測試集
test_set = []
# training_set 將剩下的字存到訓練集中
training_set = []


word_s = set([])

for word in brown.words():
    if word.isalpha():
        word = word.lower()
        if word not in word_s:
            word_s.add(word)

word_set = list(word_s)
np.random.shuffle(word_set)

test_set = word_set[:1000]
training_set = word_set[1000:]

print(len(word_set))
print(len(test_set))
print(len(training_set))

40234
1000
39234


這裡一來就不是一個人寫一個字讓對方猜了，而是電腦和人類的對決：

In [5]:
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

開始猜字遊戲，提示： _ _ _ _ _ _ 長度為 6
你還有 2 次機會。
請輸入你要猜的字：
q
你猜了： q
抱歉，再猜猜
你還有 1 次機會。
請輸入你要猜的字：
z
你猜了： z
抱歉，再猜猜
沒有機會了，正確字是 umpire


# Day 26 三種AI猜字方法

## 第一種猜字方法：隨機猜字

為了設下一個基準，我們先設計一種*AI*方法--每次從26個字母中隨機選取一個字母來猜。這裡我先將26個字母存到 `list` 中，再用 `numpy.random.choice` 隨機選取。

`test_guesser` 是用來測試平均猜錯次數的方法。

In [4]:
def test_guesser(guesser, test=test_set):
    """
        這個方法是用來測試平均猜錯的次數。
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [5]:
import string

def random_guesser(mask, guessed, **kwargs):
    """
        隨機猜字
    """

    alphabets = []
    for letter in range(97,123):
        if chr(letter) not in guessed:
            alphabets.append(chr(letter))

    picked = np.random.choice(alphabets)
    return picked

# 若要看看機器是怎麼猜字的，可以把下面這句打開
#hangman(np.random.choice(test_set), random_guesser, 10, True)

result = test_guesser(random_guesser)
print()
print("平均錯誤次數：", result)


平均錯誤次數： 16.453


## 第二種猜法：Unigram Guesser

我們可以嘗試用*Unigram*模型來訓練。我們需要知道每個字母的出現頻率，接著照出現頻率的高低來進行猜字。每當猜完一個字之後就應該把猜過的字去掉。

In [6]:
from collections import Counter

# unigram_counts 儲存了整個訓練及中每個字母的出現次數
unigram_counts = Counter()

for word in training_set:
    for letter in word:
        unigram_counts[letter] += 1

print(unigram_counts)


def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):
    """
        這個方法實作了Unigram Guesser，會根據Unigram Model每次回傳一個要猜的字。
    """
    
    unigram_keys = []

    # 照出現頻率將字母排序
    for i in range(len(unigram_counts)):
        unigram_keys.append(unigram_counts.most_common()[i][0])

    # 將猜過的字去除
    for letter in guessed:
        if letter in unigram_keys:
            unigram_keys.remove(letter)

    return unigram_keys[0]

#hangman(np.random.choice(test_set), unigram_guesser, 10, True)

result = test_guesser(unigram_guesser)
print()
print("平均猜錯次數：", result)

Counter({'e': 35207, 'i': 26008, 'a': 24361, 's': 23721, 'n': 22658, 'r': 22054, 't': 20665, 'o': 19167, 'l': 16670, 'c': 12626, 'd': 12267, 'u': 10185, 'm': 8668, 'g': 8609, 'p': 8563, 'h': 7342, 'b': 5724, 'y': 5363, 'f': 4216, 'v': 3442, 'k': 2978, 'w': 2792, 'z': 1040, 'x': 835, 'j': 641, 'q': 531})

平均猜錯次數： 10.398


## 第三種猜法：根據文字長度猜字

從和昨天[同一篇文章](http://datagenetics.com/blog/april12012/index.html)中我們看到，不同的文字長度，每個字母出現的頻率不盡相同，例如，短的字比較不會出現前綴或後綴。在這裡，我們針對不同的文字長度設計不一樣的猜字順序。

In [7]:
from collections import defaultdict

# unigram_counts_by_length 將文字長度和字母頻率map在一起
unigram_counts_by_length = defaultdict(Counter)

# 幫每一種文字長度寫不同的Unigram Model
for word in training_set:    
    this_count = Counter()
    for letter in word:
        this_count[letter] += 1
        unigram_counts_by_length[len(word)] += this_count
        this_count = Counter()

        
def exclude_guessed_letters(length_model, guessed):
    unigram_keys_by_length = []
    # 照出現頻率將字母排序
    for i in range(len(unigram_counts_by_length[length_model])):
        unigram_keys_by_length.append(unigram_counts_by_length[length_model].most_common()[i][0])
    
    # 將猜過的字去除
    for letter in guessed:
        if letter in unigram_keys_by_length:
            unigram_keys_by_length.remove(letter)
    
    return unigram_keys_by_length


lengths = sorted(unigram_counts_by_length.keys())
max_length = lengths[-1] + 1

print(unigram_counts_by_length)

def unigram_length_guesser(mask, guessed, counts=unigram_counts_by_length):
    
    length_model = len(mask)
    # 若要猜的文字長度不在unigram model時，我們將一長度來猜。
    while length_model not in lengths:
        length_model -= 1
    
    unigram_keys_by_length = exclude_guessed_letters(length_model, guessed)
    
    # 若這個文字長度沒有猜字選項了，從附近的文字長度找
    while len(unigram_keys_by_length) == 0:
        if length_model < 20:
            length_model += 1
        else:
            length_model -= 1
        unigram_keys_by_length = exclude_guessed_letters(length_model, guessed)
    
    return unigram_keys_by_length[0]


#hangman(np.random.choice(test_set), unigram_length_guesser, 10, True)

result = test_guesser(unigram_length_guesser)
print()
print("平均猜錯次數：", result)

defaultdict(<class 'collections.Counter'>, {3: Counter({'a': 238, 'e': 198, 'o': 180, 'i': 130, 'n': 120, 't': 116, 's': 115, 'p': 106, 'm': 103, 'u': 102, 'd': 100, 'r': 97, 'b': 89, 'l': 79, 'h': 78, 'g': 78, 'y': 76, 'w': 72, 'c': 70, 'f': 51, 'k': 32, 'j': 28, 'v': 25, 'x': 23, 'z': 10, 'q': 6}), 7: Counter({'e': 5324, 'a': 3539, 's': 3471, 'r': 3413, 'i': 3332, 'n': 3049, 't': 2663, 'o': 2500, 'l': 2475, 'd': 2051, 'c': 1660, 'u': 1506, 'g': 1500, 'm': 1234, 'p': 1222, 'h': 1098, 'b': 942, 'y': 749, 'f': 675, 'k': 578, 'w': 524, 'v': 464, 'z': 154, 'x': 107, 'j': 95, 'q': 83}), 9: Counter({'e': 5401, 'i': 3903, 'n': 3468, 's': 3442, 'a': 3432, 'r': 3235, 't': 3067, 'o': 2606, 'l': 2301, 'c': 1911, 'd': 1858, 'u': 1427, 'g': 1361, 'm': 1241, 'p': 1147, 'h': 1027, 'b': 756, 'y': 643, 'f': 578, 'v': 530, 'k': 368, 'w': 367, 'z': 133, 'x': 122, 'q': 85, 'j': 69}), 6: Counter({'e': 4251, 'a': 2697, 's': 2597, 'r': 2535, 'i': 2040, 'n': 1999, 'o': 1974, 'l': 1958, 't': 1856, 'd': 1620, 

## Day 28: 猜字加強版 -- Bigram Guesser

除了考慮字母出現的機率和在各種長度中各個字母出現的機率，今天我們也把字母的排列順序列入考量。例如，我們看到一個字 `m _ s s`，我們知道他有很高的機率會是母音，因此，`a`, `e`, `i`, `o`, `u`就是我們下一個猜測的目標。

在這裡，我們開發一個Bigram的語言模型。在開始訓練這個模型之前，有幾件事需要注意：(1) 要記得加上開頭標籤 `$`；(2) 要記得針對Bigram以上的模型設計對應的平滑方法。在這裡我們決定使用 *linear interpolation （線性插值法）*來smooth高元和低元模型，而插值法中要用到的 \lambda 值我們設為0.75（建議在0.7~0.8之間）。

我們猜格子的順序會從最左邊開始猜，因此最開始會在p($)的條件下猜字母。猜的過程中可能第一個字還沒猜到，後面有些就先被猜到了，等我們終於猜到第一個字母後，我們就找下一個「最左邊還沒有被猜到的字」。

In [8]:
from operator import itemgetter

bigram_counts = defaultdict(Counter) # 這個dict的key是各個字母，value是可能接在這個字母後面之字母的Counter

bigram_inter = defaultdict(list) # 根據interpolation後的機率，將接續在一個字母後面的可能字母照出現機率排序

# 開始儲存Bigram model
for word in training_set:
    this_count = Counter()
    word = '$' + word # 字首加上開頭標籤
    for i, letter in enumerate(word):
        if i+1 != len(word):
            this_count[word[i+1]] += 1
            bigram_counts[letter] += this_count
            this_count = Counter()

set_lambda = 0.75 # 設定interpolation的lambda

# Bigram Interpolated Model: p = lambda*p(w_i|w_(i-1)) + (1-lambda)*p(w_i)
# p(w_i|w_(i-1)) = count(w_i and w_(i-1)) / count(w_(i-1))
# p(w_i) = count(w_i) / sigma(count(w_i))
for key in bigram_counts.keys():
    sigma_count_wi = sum(unigram_counts.values()) # sigma(count(w_i))
    count_wi1 = sum(bigram_counts[key].values()) # count(w_(i-1))

    # 計算p(w_i|w_(i-1))和p(w_i)
    prob_of_letter = {}
    for letter in range(97,123):
        p_wi_wi1 = bigram_counts[key][chr(letter)] / count_wi1 # p(w_i|w_(i-1))
        p_wi = unigram_counts[chr(letter)] / sigma_count_wi # p(w_i)
        prob_of_letter[chr(letter)] = set_lambda*p_wi_wi1 + (1-set_lambda)*p_wi

    # 將接續在一個字母後面的可能字母照出現機率排序
    this_list = []
    for i in range(26):
        this_list.append(sorted(prob_of_letter.items(), key=itemgetter(1), reverse=True)[i][0])
    bigram_inter[key] = this_list
    

def bigram_guesser(mask, guessed, counts=bigram_counts): # add extra default arguments if needed
    """
        實現Bigram Guesser的方法，根據使用了線性插值法的Bigram model，回傳猜測的字母。
    """
    
    mask = ['$'] + mask
    w_i_1 = "" # 找到最左邊非 '_' 的字母 -> w_(i-1)
    # 找w_i_1
    for i in range(len(mask)):
        if mask[i] == '_':
            w_i_1 = mask[i-1]
            break
    copy_bi_model = bigram_inter[w_i_1].copy()
    
    # 將已經猜過的字母去除
    for letter in guessed:
        if letter in copy_bi_model:
            copy_bi_model.remove(letter)

    return copy_bi_model[0]


#hangman(np.random.choice(test_set), bigram_guesser, 26, True)

result = test_guesser(bigram_guesser)
print()
print("平均猜錯次數：", result)


平均猜錯次數： 9.054
